# Change Feed with Azure Functions
## Replication for Multiple Partition Keys

The `cart` container is partitioned on **cartId** with **TTL** enabled.

Create the `cartByItem` container partitioned on **item**.

Create two documents in the `cart` container.

We will use the change feed (triggered by Azure Functions) to keep the `cartByItem` container in sync with the `cart` container.

In [1]:
import uuid
import azure.cosmos
from azure.cosmos import PartitionKey

# Create the cartByItem container partitioned on /item
database = cosmos_client.get_database_client('acme-webstore')
cartByItemContainer = database.create_container('cartByItem', PartitionKey(path="/item"))
print('Created cartByItem container')

# Add two cart documents
cartContainer = database.get_container_client('cart')
cartContainer.create_item(body={
    "id": str(uuid.uuid4()),
    "cartId": "123",
    "item": "Surface Pro",
    "quantity": 2
})
cartContainer.create_item(body={
    "id": str(uuid.uuid4()),
    "cartId": "123",
    "item": "Surface Book",
    "quantity": 8
})
print('Created two cart documents')

Created cartByItem container
Created two cart documents


The `cartByItem` container is empty.

It will sync up to the `cart` container using an Azure Function to consume the change feed with `StartFromBeginning = true`.

In [2]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.46 queryCompileTimeInMs : 0.11 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.07 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2 retrievedDocumentSize : 613 outputDocumentCount : 2 outputDocumentSize : 239 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8


In [3]:
%%sql --database acme-webstore --container cartByItem
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.41 queryCompileTimeInMs : 0.14 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.02 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 0 retrievedDocumentSize : 0 outputDocumentCount : 0 outputDocumentSize : 49 writeOutputTimeInMs : 0 averageIndexUtilizationRation : 0

""


**Start the Replication Microservice**

Need to start from beginning to pick up the previous two documents

In [4]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.46 queryCompileTimeInMs : 0.13 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.07 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.05 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2 retrievedDocumentSize : 613 outputDocumentCount : 2 outputDocumentSize : 239 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8


In [5]:
%%sql --database acme-webstore --container cartByItem
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.43 queryCompileTimeInMs : 0.11 queryLogicalPlanBuildTimeInMs : 0.04 queryPhysicalPlanBuildTimeInMs : 0.04 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.06 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2 retrievedDocumentSize : 631 outputDocumentCount : 2 outputDocumentSize : 239 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8


Synchronize insert

In [6]:
import uuid

# Add a third cart document
cartDocument = {
    "id": str(uuid.uuid4()),
    "cartId": "def",
    "item": "Surface Pro",
    "quantity": 3
}
database = cosmos_client.get_database_client('acme-webstore')
cartContainer = database.get_container_client('cart')
cartContainer.create_item(body=cartDocument)
print('Created cart document')

Created cart document


In [7]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 1.16 queryCompileTimeInMs : 0.26 queryLogicalPlanBuildTimeInMs : 0.04 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.09 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.06 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 3 retrievedDocumentSize : 919 outputDocumentCount : 3 outputDocumentSize : 334 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8
2,879ee317-4ddd-4cb8-84d4-d8702e50444a,def,Surface Pro,3


In [8]:
%%sql --database acme-webstore --container cartByItem
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.42 queryCompileTimeInMs : 0.12 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.06 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 3 retrievedDocumentSize : 946 outputDocumentCount : 3 outputDocumentSize : 334 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8
2,879ee317-4ddd-4cb8-84d4-d8702e50444a,def,Surface Pro,3


Synchronize update

In [9]:
# Modify the document (change quantity from 3 to 4)
cartDocument['quantity'] = 4
cartContainer.replace_item(cartDocument['id'], cartDocument)
print('Modified cart document')

Modified cart document


In [10]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.5 queryCompileTimeInMs : 0.11 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.09 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.05 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 3 retrievedDocumentSize : 919 outputDocumentCount : 3 outputDocumentSize : 334 writeOutputTimeInMs : 0.02 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8
2,879ee317-4ddd-4cb8-84d4-d8702e50444a,def,Surface Pro,4


In [11]:
%%sql --database acme-webstore --container cartByItem
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.41 queryCompileTimeInMs : 0.12 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.06 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 3 retrievedDocumentSize : 946 outputDocumentCount : 3 outputDocumentSize : 334 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8
2,879ee317-4ddd-4cb8-84d4-d8702e50444a,def,Surface Pro,4


Synchronize delete

In [13]:
# Soft-delete the document using TTL
cartDocument['ttl'] = 10
cartContainer.replace_item(cartDocument['id'], cartDocument)
print('Soft-deleted cart document')

Soft-deleted cart document


In [19]:
%%sql --database acme-webstore --container cart
SELECT c.id, c.cartId, c.item, c.quantity, c.ttl FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.44 queryCompileTimeInMs : 0.12 queryLogicalPlanBuildTimeInMs : 0.03 queryPhysicalPlanBuildTimeInMs : 0.03 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.08 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.05 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2 retrievedDocumentSize : 613 outputDocumentCount : 2 outputDocumentSize : 239 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8


In [20]:
%%sql --database acme-webstore --container cartByItem
SELECT c.id, c.cartId, c.item, c.quantity FROM c

▶ View Query Metrics 
 
 
 
 totalExecutionTimeInMs : 0.4 queryCompileTimeInMs : 0.08 queryLogicalPlanBuildTimeInMs : 0.02 queryPhysicalPlanBuildTimeInMs : 0.02 queryOptimizationTimeInMs : 0 VMExecutionTimeInMs : 0.06 indexLookupTimeInMs : 0 documentLoadTimeInMs : 0.04 systemFunctionExecuteTimeInMs : 0 userFunctionExecuteTimeInMs : 0 retrievedDocumentCount : 2 retrievedDocumentSize : 631 outputDocumentCount : 2 outputDocumentSize : 239 writeOutputTimeInMs : 0.01 averageIndexUtilizationRation : 1

,id,cartId,item,quantity
0,4bf25f67-a60c-40a6-bf0f-4b6caebb5dbd,123,Surface Pro,2
1,55eb7a73-5c01-4db2-9565-6d895bed5a9d,123,Surface Book,8
